In [1]:
#!/usr/bin/env python
import wuml


##	We generated a synthetic data for regression with 4 dimensions where
##	x1 x2 has positive influence
##	x3 has no influence
##	x4 has negative influence
##
##	x1 has normal distribution
##	x2 is exponential distribution but minus 2 so it could be negative
##	x3 is uniform but shouldn't matter
##	x4 is categorical distribution.


data = wuml.wData(xpath='../../data/shap_regress_example_mix_distributions.csv', batch_size=20, 
					label_type='continuous', label_column_name='label', row_id_with_label=0)



#	Example 1
EXP = wuml.explainer(data, 	loss='mse',		# This will create a network for regression and explain instance wise 
						networkStructure=[(100,'relu'),(100,'relu'),(1,'none')], 
						max_epoch=150, learning_rate=0.001, print_network_training_status=False)

# Show the explanation results
explanation = EXP(data)	# outputs the weight importance
print('Notice that since x1 and x2 can be negative, we get both negative and positive influence.')
print('This is not correct since they should only have positive influence.')
print(explanation)



Network Info:
	Learning rate: 0.001
	Max number of epochs: 150
	Cost Function: mse
	Train Loop Callback: None
	Cuda Available: True
	Network Structure
		Linear(in_features=4, out_features=100, bias=True) , relu
		Linear(in_features=100, out_features=100, bias=True) , relu
		Linear(in_features=100, out_features=1, bias=True) , none



  0%|          | 0/30 [00:00<?, ?it/s]

Notice that since x1 and x2 can be negative, we get both negative and positive influence.
This is not correct since they should only have positive influence.
[[  5.5714  -0.608    0.8444 -37.7602]
 [  1.696    1.258    0.2566   0.    ]
 [  3.81    -1.1233   0.7506 -41.8313]
 [  8.516    0.6756   1.1793 -11.7473]
 [  6.329   -0.0741   1.0796 -40.7762]
 [ -3.9921  -3.0936   0.2372 -39.5582]
 [  4.2476   0.773    1.2367 -13.0493]
 [ -0.4727  -0.86     0.0213 -39.7116]
 [ -0.3629  -2.52     0.1488 -40.6869]
 [  1.513   -2.0998   0.0921 -40.9965]
 [  0.6937   0.4914   0.4612 -12.5117]
 [  4.9272  -1.0815   0.1559 -41.2271]
 [  2.9545  -0.0215   0.4478 -27.6767]
 [  0.4126  -1.6826   0.1907 -41.1262]
 [  1.1429  -3.0555   0.0392 -32.9816]
 [  1.6646   0.3132   0.9186 -12.5198]
 [  5.5249  -1.1836   0.7384 -41.4673]
 [ -1.0058   0.0345   0.3856 -12.3271]
 [  1.0319  -0.6716   0.2569 -41.1145]
 [ -3.1466  -2.8786   0.0401 -39.3353]
 [-11.2182  -2.0143  -0.0569 -37.3623]
 [  2.602   -1.4833   0

In [2]:
#	Example 2
Cdata = wuml.center_and_scale(data)
EXP2 = wuml.explainer(Cdata, 	loss='mse',		# This will create a network for regression and explain instance wise 
						networkStructure=[(100,'relu'),(100,'relu'),(1,'none')], 
						max_epoch=150, learning_rate=0.001, print_network_training_status=False)

# Show the explanation results
explanation = EXP2(Cdata)	# outputs the weight importance
print('Notice that centering does not help since x1, x2, x4 can be both positive and negative, which is not correct.')
print(explanation)

Network Info:
	Learning rate: 0.001
	Max number of epochs: 150
	Cost Function: mse
	Train Loop Callback: None
	Cuda Available: True
	Network Structure
		Linear(in_features=4, out_features=100, bias=True) , relu
		Linear(in_features=100, out_features=100, bias=True) , relu
		Linear(in_features=100, out_features=1, bias=True) , none



  0%|          | 0/30 [00:00<?, ?it/s]

Notice that centering does not help since x1, x2, x4 can be both positive and negative, which is not correct.
[[ -0.8643   4.4328  -3.1433 -11.62  ]
 [ -0.1604   1.8543  -2.9471  22.8934]
 [  0.832   -3.0174  -6.4843 -12.1667]
 [  0.2336  -1.1947  -0.1282  17.9027]
 [ -0.6752   3.2985  -7.1346 -12.0999]
 [ -6.7101  -8.1003   0.1931 -12.5448]
 [  1.2718   2.0287  -3.5312  14.9719]
 [ -2.6718   2.8608  -7.9017 -13.4347]
 [ -2.1916  -5.4622  -1.9146 -14.3809]
 [ -0.1149  -6.3149  -4.6145 -14.4522]
 [ -1.5148  -3.0222  -0.8517  17.4442]
 [  2.145   -2.9525  -4.5327 -14.2326]
 [  0.978    0.8465  -1.2136   5.8853]
 [ -1.2423  -1.8821  -0.1427 -14.6745]
 [  0.0028  -1.3158  -5.0155 -11.0955]
 [ -0.4376  -5.6956  -2.5139  16.3149]
 [  1.2316  -5.4971  -0.1597 -14.0195]
 [ -3.6907  -5.1446  -1.2996  17.4586]
 [ -0.4266   2.7146   0.2481 -14.4017]
 [ -6.3501  -4.4903  -5.8334 -13.3131]
 [-20.2453  -1.2556   0.6777 -11.3635]
 [  0.4882  -4.8477  -4.7376 -12.3253]
 [  1.3422  -6.5227  -1.0975 -14

In [3]:
#	Example 3
Udata = wuml.use_reverse_cdf_to_map_data_between_0_and_1(data, output_type_name='wData')
EXP3 = wuml.explainer(Udata, 	loss='mse',		# This will create a network for regression and explain instance wise 
						networkStructure=[(600,'relu'),(600,'relu'),(600,'relu'),(1,'none')], 
						max_epoch=600, learning_rate=0.001, print_network_training_status=False)

# Show the regression results
Ŷ = EXP3.net(Udata, output_type='ndarray')
SR_train = wuml.summarize_regression_result(Udata.Y, Ŷ)
print(SR_train.true_vs_predict())

# Show the explanation results
explanation = EXP3(Udata)	# outputs the weight importance
print('If we map all the data into the range of [0,1], notice that x1, x2, x4 all have the correct attribution sign')
print('We noticed that it requires a larger network, and longer training time, but it gives better attributions.')
print(explanation)


Network Info:
	Learning rate: 0.001
	Max number of epochs: 600
	Cost Function: mse
	Train Loop Callback: None
	Cuda Available: True
	Network Structure
		Linear(in_features=4, out_features=600, bias=True) , relu
		Linear(in_features=600, out_features=600, bias=True) , relu
		Linear(in_features=600, out_features=600, bias=True) , relu
		Linear(in_features=600, out_features=1, bias=True) , none

Avg error: 0.0547

['y' 'ŷ']
[-30.88 -31.  ]
[  0.9    0.95]
[-40.3  -40.22]
[ -3.22  -3.14]
[-35.01 -34.83]
[-47.01 -46.8 ]
[ -6.81  -6.77]
[-43.36 -43.35]
[-43.91 -43.93]
[-42.48 -42.45]
[-11.22 -11.2 ]
[-38.34 -38.37]
[-22.26 -22.23]
[-42.91 -42.89]
[-36.14 -36.12]
[-10.87 -10.84]
[-38.87 -38.96]
[-12.38 -12.34]
[-41.63 -41.61]
[-46.63 -46.64]
[-52.6  -52.56]
[-41.44 -41.48]
[-41.15 -41.09]
[-46.21 -46.25]
[-33.74 -33.72]
[-48.25 -48.3 ]
[-25.57 -25.53]
[-44.15 -44.2 ]
[  6.88   6.97]
[ -7.29  -7.21]



  0%|          | 0/30 [00:00<?, ?it/s]

If we map all the data into the range of [0,1], notice that x1, x2, x4 all have the correct attribution sign
We noticed that it requires a larger network, and longer training time, but it gives better attributions.
[[ 13.7853   1.092   -0.9513 -43.5258]
 [  7.2627  -2.742    0.4676  -2.6335]
 [  8.5319  -1.4158  -0.8984 -45.0424]
 [ 11.4175  -1.1966   0.9138 -12.8759]
 [ 12.6628  -0.3036  -1.5066 -44.2844]
 [  2.3451  -2.1981   0.3938 -45.9433]
 [ 10.892   -2.2991  -0.8539 -13.1138]
 [  4.8875  -2.189    0.0246 -44.6758]
 [  4.9128  -2.0576   0.867  -46.2521]
 [  5.8633  -1.3533   0.5028 -46.0646]
 [  5.6752  -2.7816   1.2726 -13.9708]
 [  8.9038  -0.8214   0.3208 -45.369 ]
 [  7.3389  -2.7005   0.0387 -25.505 ]
 [  6.2463  -2.4337   0.5396 -45.8414]
 [  8.0354   0.7367   0.0338 -43.5303]
 [  6.5469  -2.0901   0.0591 -13.9546]
 [  8.7763  -1.1814   0.4605 -45.6122]
 [  4.4612  -2.6167   1.2697 -14.0512]
 [  7.3768  -2.408   -0.0843 -45.0906]
 [  2.5573  -2.3905   0.3641 -45.7756]
 [  0